In [1]:
#run to start
curr_dir = f'/user_data/csimmon2/git_repos/ptoc'

import sys
sys.path.insert(0,curr_dir)
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import scipy.stats as stats
import scipy
import statsmodels.api as s
from sklearn import metrics

import os
import pandas as pd
import numpy as np
from nilearn import image, maskers, plotting, datasets
from nilearn.maskers import NiftiMasker
from nilearn.datasets import load_mni152_brain_mask
from nilearn.glm.first_level import compute_regressor
import nibabel as nib
import sys
import time
import itertools 
import warnings


import pdb
import ptoc_params as params

from plotnine import *
#from plotnine import ggplot, aes, geom_point

#hide warnings
import warnings
warnings.filterwarnings('ignore')

#load additional libraries
from nilearn import image, plotting, input_data, glm
from nilearn.input_data import NiftiMasker
import nibabel as nib
import statsmodels.api as sm
from nilearn.datasets import load_mni152_brain_mask, load_mni152_template
from nilearn.glm.first_level import compute_regressor ##?????

data_dir = params.data_dir
results_dir = params.results_dir
fig_dir = params.fig_dir
raw_dir = params.raw_dir
sub_info = params.sub_info
task_info = params.task_info

suf = params.suf
#mni = load_mni152_brain_mask()

'''exp info'''
#load subject info
sub_info = pd.read_csv(f'{curr_dir}/sub_info.csv')
#subs = sub_info[sub_info['group'] == 'control']['sub'].tolist()


##Just controls
#subs = sub_info[sub_info['group'] == 'control']['sub'].tolist()
study = 'ptoc'
study_dir = f"/lab_data/behrmannlab/vlad/{study}"
results_dir = '/user_data/csimmon2/GitHub_Repos/ptoc/results'
exp = ''
#rois = ['LO']  # Run for one ROI initially
#rois = ['LO', 'PFS', 'pIPS','aIPS', 'V1']
control_tasks = ['loc']
file_suf = ''

'''scan params'''
tr = 2 #ptoc_params
vols = 184 #ptoc_params

whole_brain_mask = load_mni152_brain_mask()
mni = load_mni152_template()
brain_masker = NiftiMasker(whole_brain_mask, smoothing_fwhm=0, standardize=True)

In [4]:
import os
import pandas as pd
import numpy as np
from nilearn import image, plotting
import matplotlib.pyplot as plt
import itertools
import nibabel as nib

study = 'ptoc'
study_dir = f"/lab_data/behrmannlab/vlad/{study}"
curr_dir = f'/user_data/csimmon2/git_repos/ptoc'
mni_parcel_dir = f'{curr_dir}/roiParcels'

def extract_roi_coords():
    parcels = ['pIPS', 'LO']
    hemispheres = ['left', 'right']
    sub_info = pd.read_csv(f'{curr_dir}/sub_info.csv')
    subs = sub_info[sub_info['group'] == 'control']['sub'].tolist()
    #subs = ['sub-025']
    runs = [1, 2, 3]
    run_combos = list(itertools.combinations(runs, 2))
    
    # Load MNI152 template for visualization
    mni_template = image.load_img('/opt/fsl/6.0.3/data/standard/MNI152_T1_2mm_brain.nii.gz')
    
    for ss in subs:
        print(f'Processing subject: {ss}')
        sub_dir = f'{study_dir}/{ss}/ses-01'
        roi_dir = f'{sub_dir}/derivatives/rois'
        os.makedirs(f'{roi_dir}/spheres', exist_ok=True)
        exp_dir = f'{sub_dir}/derivatives/fsl'
        roi_coords = pd.DataFrame(columns=['index', 'task', 'roi', 'hemisphere', 'x', 'y', 'z'])
        
        index = 0
        for rc in run_combos:
            print(f"Run combination: {rc}")
            all_runs = []
            for rn in rc:
                curr_run_path = f'/lab_data/behrmannlab/vlad/hemispace/{ss}/ses-01/derivatives/fsl/loc/run-0{rn}/1stLevel.feat/stats/zstat3_mni.nii.gz'
                if os.path.exists(curr_run_path):
                    curr_run = image.load_img(curr_run_path)
                    all_runs.append(curr_run)
                else:
                    print(f'File does not exist: {curr_run_path}')
            
            if len(all_runs) != 2:
                print(f"Skipping combination {rc} due to missing data")
                continue
            
            mean_zstat = image.mean_img(all_runs)
            
            for pr in parcels:
                roi_path = f'{mni_parcel_dir}/{pr}.nii.gz'
                if os.path.exists(roi_path):
                    roi = image.load_img(roi_path)
                    
                    for hemi in hemispheres:
                        # Create hemisphere mask
                        hemi_mask = np.zeros(roi.shape)
                        if hemi == 'left':
                            hemi_mask[:roi.shape[0]//2, :, :] = 1
                        else:
                            hemi_mask[roi.shape[0]//2:, :, :] = 1
                        hemi_mask_img = nib.Nifti1Image(hemi_mask, roi.affine)
                        
                        # Apply hemisphere mask to ROI
                        masked_roi = image.math_img('img1 * img2', img1=roi, img2=hemi_mask_img)
                        
                        # Find peak voxel within hemispheric ROI
                        masked_data = image.math_img('img1 * img2', img1=mean_zstat, img2=masked_roi).get_fdata()
                        peak_idx = np.unravel_index(np.argmax(masked_data), masked_data.shape)
                        peak_coords = image.coord_transform(*peak_idx, mean_zstat.affine)
                        
                        roi_coords = pd.concat([roi_coords, pd.DataFrame({
                            'index': [index],
                            'task': ['loc'],
                            'roi': [pr],
                            'hemisphere': [hemi],
                            'x': [peak_coords[0]],
                            'y': [peak_coords[1]],
                            'z': [peak_coords[2]]
                        })], ignore_index=True)
                        
                        # Visualize ROI and peak voxel on MNI template
                        display = plotting.plot_roi(masked_roi, bg_img=mni_template,
                                                    title=f'{ss} - {pr} - {hemi} - Runs {rc[0]}{rc[1]}')
                        display.add_markers([peak_coords], marker_color='r', marker_size=100)
                        #!SECTIONplt.savefig(f'{roi_dir}/{ss}_{pr}_{hemi}_runs{rc[0]}{rc[1]}_peak_mni.png')
                        plt.close()
                        
                        print(f"ROI: {pr}, Hemisphere: {hemi}, Peak coordinates: {peak_coords}")
                else:
                    print(f'ROI file does not exist: {roi_path}')
            
            index += 1
        
        roi_coords.to_csv(f'{roi_dir}/spheres/sphere_coords_std.csv', index=False)
        print(f"ROI coordinates saved to {roi_dir}/spheres/sphere_coords_std.csv")

if __name__ == "__main__":
    extract_roi_coords()
    print("ROI coordinate extraction completed.")

Processing subject: sub-025
Run combination: (1, 2)
ROI: pIPS, Hemisphere: left, Peak coordinates: (42.0, -74.0, 14.0)
ROI: pIPS, Hemisphere: right, Peak coordinates: (-42.0, -76.0, 30.0)
ROI: LO, Hemisphere: left, Peak coordinates: (44.0, -74.0, -10.0)
ROI: LO, Hemisphere: right, Peak coordinates: (-48.0, -78.0, -12.0)
Run combination: (1, 3)
ROI: pIPS, Hemisphere: left, Peak coordinates: (44.0, -74.0, 14.0)
ROI: pIPS, Hemisphere: right, Peak coordinates: (-22.0, -74.0, 32.0)
ROI: LO, Hemisphere: left, Peak coordinates: (44.0, -74.0, -10.0)
ROI: LO, Hemisphere: right, Peak coordinates: (-48.0, -78.0, -12.0)
Run combination: (2, 3)
ROI: pIPS, Hemisphere: left, Peak coordinates: (44.0, -74.0, 14.0)
ROI: pIPS, Hemisphere: right, Peak coordinates: (-22.0, -76.0, 34.0)
ROI: LO, Hemisphere: left, Peak coordinates: (44.0, -74.0, -10.0)
ROI: LO, Hemisphere: right, Peak coordinates: (-46.0, -76.0, -10.0)
ROI coordinates saved to /lab_data/behrmannlab/vlad/ptoc/sub-025/ses-01/derivatives/rois/s

In [ ]:
##8/9/24 std space extract_roi_coords

study = 'ptoc'
study_dir = f"/lab_data/behrmannlab/vlad/{study}"
curr_dir = f'/user_data/csimmon2/git_repos/ptoc'
mni_parcel_dir = f'{curr_dir}/roiParcels'

def extract_roi_coords():
    parcels = ['pIPS', 'LO']
    sub_info = pd.read_csv(f'{curr_dir}/sub_info.csv')
    subs = sub_info[sub_info['group'] == 'control']['sub'].tolist()
    #subs = ['sub-025']
    runs = [1, 2, 3]
    run_combos = list(itertools.combinations(runs, 2))
    
    # Load MNI152 template for visualization
    mni_template = image.load_img('/opt/fsl/6.0.3/data/standard/MNI152_T1_2mm_brain.nii.gz')
    
    for ss in subs:
        print(f'Processing subject: {ss}')
        sub_dir = f'{study_dir}/{ss}/ses-01'
        roi_dir = f'{sub_dir}/derivatives/rois'
        os.makedirs(f'{roi_dir}/spheres', exist_ok=True)
        exp_dir = f'{sub_dir}/derivatives/fsl'
        roi_coords = pd.DataFrame(columns=['index', 'task', 'roi', 'x', 'y', 'z'])
        
        index = 0
        for rc in run_combos:
            print(f"Run combination: {rc}")
            all_runs = []
            for rn in rc:
                curr_run_path = f'/lab_data/behrmannlab/vlad/hemispace/{ss}/ses-01/derivatives/fsl/loc/run-0{rn}/1stLevel.feat/stats/zstat3_mni.nii.gz'
                if os.path.exists(curr_run_path):
                    curr_run = image.load_img(curr_run_path)
                    all_runs.append(curr_run)
                else:
                    print(f'File does not exist: {curr_run_path}')
            
            if len(all_runs) != 2:
                print(f"Skipping combination {rc} due to missing data")
                continue
            
            mean_zstat = image.mean_img(all_runs)
            
            for pr in parcels:
                roi_path = f'{mni_parcel_dir}/{pr}.nii.gz'
                if os.path.exists(roi_path):
                    roi = image.load_img(roi_path)
                    # Find peak voxel within ROI
                    masked_data = image.math_img('img1 * img2', img1=mean_zstat, img2=roi).get_fdata()
                    peak_idx = np.unravel_index(np.argmax(masked_data), masked_data.shape)
                    peak_coords = image.coord_transform(*peak_idx, mean_zstat.affine)
                    
                    roi_coords = pd.concat([roi_coords, pd.DataFrame({
                        'index': [index],
                        'task': ['loc'],
                        'roi': [pr],
                        'x': [peak_coords[0]],
                        'y': [peak_coords[1]],
                        'z': [peak_coords[2]]
                    })], ignore_index=True)
                    
                    # Visualize ROI and peak voxel on MNI template
                    display = plotting.plot_roi(roi, bg_img=mni_template,
                                                title=f'{ss} - {pr} - Runs {rc[0]}{rc[1]}')
                    display.add_markers([peak_coords], marker_color='r', marker_size=100)
                    #plt.savefig(f'{roi_dir}/{ss}_{pr}_runs{rc[0]}{rc[1]}_peak_mni.png')
                    plt.close()
                    
                    print(f"ROI: {pr}, Peak coordinates: {peak_coords}")
                else:
                    print(f'ROI file does not exist: {roi_path}')
            
            index += 1
        
        roi_coords.to_csv(f'{roi_dir}/spheres/sphere_coords_std.csv', index=False)
        print(f"ROI coordinates saved to {roi_dir}/spheres/sphere_coords_std.csv")

if __name__ == "__main__":
    extract_roi_coords()
    print("ROI coordinate extraction completed.")

In [ ]:
## converting filtered_func_data to MNI space - not verified.
import os
from nipype.interfaces import fsl

def convert_func_reg_to_std(subject_id, run_id):
    # Set up directories
    study_dir = f"/lab_data/behrmannlab/vlad/ptoc"
    sub_dir = f'{study_dir}/{subject_id}/ses-01'
    func_dir = f'{sub_dir}/derivatives/fsl/loc/run-0{run_id}/1stLevel.feat'
    reg_dir = f'{func_dir}/reg'
    
    # Define input and output files
    in_file = f'{func_dir}/filtered_func_data_reg.nii.gz'
    out_file = f'{func_dir}/filtered_func_data_reg_mni.nii.gz'
    
    # Set up FSL's ApplyWarp
    apply_warp = fsl.ApplyWarp()
    apply_warp.inputs.in_file = in_file
    apply_warp.inputs.ref_file = os.path.join(os.environ['FSLDIR'], 'data/standard/MNI152_T1_2mm_brain.nii.gz')
    apply_warp.inputs.out_file = out_file

    # Check for available transformation files and set up accordingly
    if os.path.exists(f'{reg_dir}/example_func2standard.nii.gz'):
        apply_warp.inputs.field_file = f'{reg_dir}/example_func2standard.nii.gz'
    elif os.path.exists(f'{reg_dir}/example_func2highres.mat') and os.path.exists(f'{reg_dir}/highres2standard.nii.gz'):
        apply_warp.inputs.field_file = f'{reg_dir}/highres2standard.nii.gz'
        apply_warp.inputs.premat = f'{reg_dir}/example_func2highres.mat'
    else:
        raise FileNotFoundError("Required transformation files not found in the reg directory.")

    # Run the warp
    result = apply_warp.run()
    
    print(f"Conversion completed. Output file: {out_file}")
    return out_file

# Example usage:
subject_id = 'sub-025'
run_id = 1

# Convert filtered_func_data_reg to standard space
std_func_reg = convert_func_reg_to_std(subject_id, run_id)

In [2]:
#8.9.24 fc_ppi attempted standard space 
import os
import pandas as pd
import numpy as np
from nilearn import image, input_data
from nilearn.maskers import NiftiMasker
from nilearn.datasets import load_mni152_brain_mask, load_mni152_template
from nilearn.glm.first_level import compute_regressor
import nibabel as nib
import sys
import itertools
import time

# Import your parameters
curr_dir = f'/user_data/csimmon2/git_repos/ptoc'
sys.path.insert(0, curr_dir)
import ptoc_params as params

# Set up directories and parameters
study = 'ptoc'
study_dir = f"/lab_data/behrmannlab/vlad/{study}"
results_dir = '/user_data/csimmon2/GitHub_Repos/ptoc/results'
raw_dir = params.raw_dir

whole_brain_mask = load_mni152_brain_mask()
mni = load_mni152_template()
brain_masker = input_data.NiftiMasker(whole_brain_mask,
    smoothing_fwhm=0, standardize=True)

#sub_info = pd.read_csv(f'{curr_dir}/sub_info.csv')
#subs = sub_info[sub_info['group'] == 'control']['sub'].tolist()
subs = ['sub-025']
rois = ['LO']
tsk = 'loc'

run_num = 3
run_combos = list(itertools.combinations(range(1, run_num + 1), 2))


def extract_roi_sphere(img, coords):
    roi_masker = input_data.NiftiSpheresMasker([tuple(coords)], radius = 6)
    seed_time_series = roi_masker.fit_transform(img)
    phys = np.mean(seed_time_series, axis= 1)
    #phys = (phys - np.mean(phys)) / np.std(phys) #TRY WITHOUT STANDARDIZING AT SOME POINT
    phys = phys.reshape((phys.shape[0],1))
    return phys

def make_psy_cov(runs, ss):
    temp_dir = f'{raw_dir}/{ss}/ses-01'
    cov_dir = f'{temp_dir}/covs'
    vols, tr = 184, 2.0
    times = np.arange(0, vols * len(runs) * tr, tr)
    full_cov = pd.DataFrame(columns=['onset', 'duration', 'value'])
    
    for rn, run in enumerate(runs):
        ss_num = ss.split('-')[1]
        obj_cov_file = f'{cov_dir}/catloc_{ss_num}_run-0{run}_Object.txt'
        scr_cov_file = f'{cov_dir}/catloc_{ss_num}_run-0{run}_Scramble.txt'
        
        if not os.path.exists(obj_cov_file) or not os.path.exists(scr_cov_file):
            print(f'Covariate file not found for run {run}')
            raise FileNotFoundError(f"Missing covariate file for run {run}")
        
        obj_cov = pd.read_csv(obj_cov_file, sep='\t', header=None, names=['onset', 'duration', 'value'])
        scr_cov = pd.read_csv(scr_cov_file, sep='\t', header=None, names=['onset', 'duration', 'value'])
        scr_cov['value'] *= -1
        
        curr_cov = pd.concat([obj_cov, scr_cov])
        curr_cov['onset'] += rn * vols * tr
        full_cov = pd.concat([full_cov, curr_cov])
    
    full_cov = full_cov.sort_values(by=['onset']).reset_index(drop=True)
    
    if full_cov['onset'].max() >= times[-1] or full_cov['onset'].min() < 0:
        raise ValueError("Event onsets are outside the expected time range")
    
    cov = full_cov.to_numpy()
    
    psy, _ = compute_regressor(cov.T, 'spm', times)
    
    if len(psy) != vols * len(runs):
        raise ValueError(f"Generated psy does not match expected length. Expected {vols * len(runs)}, got {len(psy)}")
    
    return psy

def conduct_analyses():
    for ss in subs:
        subject_start_time = time.time()
        print(f"Processing subject: {ss}")
        sub_dir = f'{study_dir}/{ss}/ses-01/'
        roi_dir = f'{sub_dir}derivatives/rois'
        temp_dir = f'{raw_dir}/{ss}/ses-01/derivatives/fsl/loc'
        exp = 'loc'
        exp_dir = f'{sub_dir}/derivatives/fsl/{exp}'
        out_dir = f'{study_dir}/{ss}/ses-01/derivatives'
        os.makedirs(f'{out_dir}/fc', exist_ok=True) #os.makedirs(f'{out_dir}/fc_ppi', exist_ok=True)
        
        roi_coords = pd.read_csv(f'{roi_dir}/spheres/sphere_coords.csv')

        for rr in rois:
            all_runs_fc = []
            all_runs_ppi = []
            
            print(f"Processing ROI: {rr}")
            fc_file = f'{out_dir}/fc/{ss}_{rr}_{tsk}_fc_std.nii.gz'
            ppi_file = f'{out_dir}/fc/{ss}_{rr}_{tsk}_ppi_std.nii.gz'
            
            do_fc = not os.path.exists(fc_file)
            do_ppi = not os.path.exists(ppi_file)
            
            if not do_fc and not do_ppi:
                print(f'Both FC and PPI files for {rr} already exist. Skipping...')
                continue
            
            for rcn, rc in enumerate(run_combos):
                curr_coords = roi_coords[(roi_coords['index'] == rcn) & 
                                            (roi_coords['task'] == tsk) & 
                                            (roi_coords['roi'] == rr)]
                
                peak_coords = curr_coords[['x', 'y', 'z']].values[0]
                held_out_run = list(set(range(1, 4)) - set(rc))[0]
                ###img4d = image.load_img(f'{exp_dir}/run-0{held_out_run}/1stLevel.feat/filtered_func_data_reg.nii.gz') ### change to std space output
                img4d = image.clean_img(img4d, standardize=True)
                
                phys = extract_roi_sphere(img4d, peak_coords)
                psy = make_psy_cov([held_out_run], ss)  # Generate psy for the current run combination
                #here is where it gets weird
                
                #combine phys (seed time series) and psy (task time series) into a regressor
                confounds = pd.DataFrame({'psy': psy[:,0], 'phys': phys[:,0]})
                
                brain_time_series_ppi = brain_masker.fit_transform(img4d, confounds=[confounds])
                brain_time_series_fc = brain_masker.fit_transform(img4d)
                
                if do_fc: # need to confirm analysis, is it psy or phys?
                    # FC Analysis
                    correlations_fc = np.dot(brain_time_series_fc.T, phys) / phys.shape[0]
                    correlations_fc = np.arctanh(correlations_fc)
                    correlation_fc_img = brain_masker.inverse_transform(correlations_fc.T)
                    all_runs_fc.append(correlation_fc_img)
                
                if do_ppi:
                    # PPI Analysis
                    ppi = phys * psy
                    ppi = ppi.reshape((ppi.shape[0],1))
                    correlations_ppi = np.dot(brain_time_series_ppi.T, ppi) / ppi.shape[0]
                    print(ss, rr, tsk, correlations_ppi.max())
                    correlations_ppi = np.arctanh(correlations_ppi)
                    correlations_ppi_img = brain_masker.inverse_transform(correlations_ppi.T)
                    all_runs_ppi.append(correlations_ppi_img)

            if do_fc:
                mean_fc = image.mean_img(all_runs_fc)
                nib.save(mean_fc, fc_file)
                print(f'Saved FC result for {rr}')
            
            if do_ppi:
                mean_ppi = image.mean_img(all_runs_ppi)
                nib.save(mean_ppi, ppi_file)
                print(f'Saved PPI result for {rr}')

# Call the function
conduct_analyses()

Processing subject: sub-025
Processing ROI: LO
Both FC and PPI files for LO already exist. Skipping...
